In [1]:
#Link: https://fifa-dashboard-render.onrender.com

# -*- coding: utf-8 -*-
"""CP321_A7.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1HPlM-rHswewaua3c_cfc_3euYXpXSV2k
"""

# Commented out IPython magic to ensure Python compatibility.
#pip install jupyter-dash
# %pip install dash

# Importing all the libraries
import pandas as pd
import dash
from dash import dcc, html, Input, Output, jupyter_dash
import plotly.express as px

import pandas as pd

world_cup_finals_data = [
    {"Year": 1930, "Winner": "Uruguay",   "RunnerUp": "Argentina", "Score": "4–2"},
    {"Year": 1934, "Winner": "Italy",     "RunnerUp": "Czechoslovakia", "Score": "2–1"},
    {"Year": 1938, "Winner": "Italy",     "RunnerUp": "Hungary",   "Score": "4–2"},
    # 1950 was decided by a final group (the decisive match was Uruguay vs. Brazil)
    {"Year": 1950, "Winner": "Uruguay",   "RunnerUp": "Brazil",    "Score": "2–1"},
    {"Year": 1954, "Winner": "Germany",   "RunnerUp": "Hungary",   "Score": "3–2"},
    {"Year": 1958, "Winner": "Brazil",    "RunnerUp": "Sweden",    "Score": "5–2"},
    {"Year": 1962, "Winner": "Brazil",    "RunnerUp": "Czechoslovakia", "Score": "3–1"},
    {"Year": 1966, "Winner": "England",   "RunnerUp": "Germany",   "Score": "4–2 (a.e.t.)"},
    {"Year": 1970, "Winner": "Brazil",    "RunnerUp": "Italy",     "Score": "4–1"},
    {"Year": 1974, "Winner": "Germany",   "RunnerUp": "Netherlands", "Score": "2–1"},
    {"Year": 1978, "Winner": "Argentina", "RunnerUp": "Netherlands", "Score": "3–1 (a.e.t.)"},
    {"Year": 1982, "Winner": "Italy",     "RunnerUp": "Germany",   "Score": "3–1"},
    {"Year": 1986, "Winner": "Argentina", "RunnerUp": "Germany",   "Score": "3–2"},
    {"Year": 1990, "Winner": "Germany",   "RunnerUp": "Argentina", "Score": "1–0"},
    {"Year": 1994, "Winner": "Brazil",    "RunnerUp": "Italy",     "Score": "0–0 (3–2 pens)"},
    {"Year": 1998, "Winner": "France",    "RunnerUp": "Brazil",    "Score": "3–0"},
    {"Year": 2002, "Winner": "Brazil",    "RunnerUp": "Germany",   "Score": "2–0"},
    {"Year": 2006, "Winner": "Italy",     "RunnerUp": "France",    "Score": "1–1 (5–3 pens)"},
    {"Year": 2010, "Winner": "Spain",     "RunnerUp": "Netherlands", "Score": "1–0 (a.e.t.)"},
    {"Year": 2014, "Winner": "Germany",   "RunnerUp": "Argentina", "Score": "1–0 (a.e.t.)"},
    {"Year": 2018, "Winner": "France",    "RunnerUp": "Croatia",   "Score": "4–2"},
    {"Year": 2022, "Winner": "Argentina", "RunnerUp": "France",    "Score": "3–3 (4–2 pens)"}
]

for row in world_cup_finals_data:
    if row["Winner"] in ["West Germany"]:
        row["Winner"] = "Germany"
    if row["RunnerUp"] in ["West Germany"]:
        row["RunnerUp"] = "Germany"

df = pd.DataFrame(world_cup_finals_data)

# Display or save to CSV for your Dash app
print(df)

win_counts = df["Winner"].value_counts().reset_index()
win_counts.columns = ["Country", "Wins"]

country_iso_map = {
    "Brazil": "BRA",
    "Germany": "DEU",
    "Italy": "ITA",
    "Argentina": "ARG",
    "France": "FRA",
    "Uruguay": "URY",
    "England": "GBR",
    "Spain": "ESP",
    "Netherlands": "NLD",
    "Croatia": "HRV",
    "Sweden": "SWE",
    "Hungary": "HUN",
    "Czechoslovakia": "CZE"
}
win_counts["ISO_Code"] = win_counts["Country"].map(country_iso_map)

app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H1("FIFA World Cup Winners Dashboard", style={"textAlign": "center"}),

    dcc.Graph(id="choropleth-map"),

    html.Label("Select a Country:", style={"marginTop": "20px"}),
    dcc.Dropdown(
        options=[{"label": country, "value": country} for country in sorted(win_counts["Country"])],
        id="country-dropdown",
        placeholder="Select a country"
    ),
    html.Div(id="country-output", style={"marginTop": "10px", "fontWeight": "bold"}),

    html.Label("Select a Year:", style={"marginTop": "20px"}),
    dcc.Dropdown(
        options=[{"label": year, "value": year} for year in sorted(df["Year"], reverse=True)],
        id="year-dropdown",
        placeholder="Select a year"
    ),
    html.Div(id="year-output", style={"marginTop": "10px", "fontWeight": "bold"})
])

@app.callback(
    Output("choropleth-map", "figure"),
    Input("country-dropdown", "value")
)
def update_map(selected_country):
    fig = px.choropleth(
        win_counts,
        locations="ISO_Code",
        color="Wins",
        hover_name="Country",
        color_continuous_scale="Reds",
        title="World Cup Wins by Country"
    )
    fig.update_layout(geo=dict(showframe=False, showcoastlines=False))
    return fig

@app.callback(
    Output("country-output", "children"),
    Input("country-dropdown", "value")
)
def display_country_wins(country):
    if not country:
        return ""
    wins = win_counts.loc[win_counts["Country"] == country, "Wins"].values[0]
    return f"{country} has won the FIFA World Cup {wins} time(s)."

@app.callback(
    Output("year-output", "children"),
    Input("year-dropdown", "value")
)
def display_year_results(year):
    if not year:
        return ""
    row = df[df["Year"] == year].iloc[0]
    return f"In {year}, the Winner was {row['Winner']} and the Runner-Up was {row['Runner-Up']}."

if __name__ == "__main__":
    app.run(debug=True)

    Year     Winner        RunnerUp           Score
0   1930    Uruguay       Argentina             4–2
1   1934      Italy  Czechoslovakia             2–1
2   1938      Italy         Hungary             4–2
3   1950    Uruguay          Brazil             2–1
4   1954    Germany         Hungary             3–2
5   1958     Brazil          Sweden             5–2
6   1962     Brazil  Czechoslovakia             3–1
7   1966    England         Germany    4–2 (a.e.t.)
8   1970     Brazil           Italy             4–1
9   1974    Germany     Netherlands             2–1
10  1978  Argentina     Netherlands    3–1 (a.e.t.)
11  1982      Italy         Germany             3–1
12  1986  Argentina         Germany             3–2
13  1990    Germany       Argentina             1–0
14  1994     Brazil           Italy  0–0 (3–2 pens)
15  1998     France          Brazil             3–0
16  2002     Brazil         Germany             2–0
17  2006      Italy          France  1–1 (5–3 pens)
18  2010    